# Training with RLlib PPO
[RLlib](https://docs.ray.io/en/latest/rllib/index.html) is a high-performance, distributed
reinforcement learning library. It is preferable to other RL libraries (e.g. Stable Baselines
3) for `bsk_rl` environments because it steps environments copies asynchronously; because 
of the variable step lengths, variable episode step counts, and long episode reset times, 
stepping each environment independently can increase step throughput by 2-5 times.

<div class="alert alert-warning">

**Warning:** RLlib had a bug that results in an undesirable timeout which stops
training. It has since been resolved: https://github.com/ray-project/ray/pull/45147

</div>

RLlib is actively developed and can change significantly from version to version. For this
script, the following version is used:

In [1]:
from importlib.metadata import version
version("ray")  # Parent package of RLlib

'2.35.0'

## Define the Environment
A nadir-scanning environment is created, to the one used in [this paper](https://hanspeterschaub.info/Papers/Stephenson2024.pdf). 
The satellite has to collect data while managing the data buffer level and battery level.

First, the satellite class is defined. A custom dynamics model is created that defines
a few additional properties to use in the state.

In [2]:
import numpy as np
from bsk_rl import act, data, obs, sats, scene
from bsk_rl.sim import dyn, fsw

class ScanningDownlinkDynModel(dyn.ContinuousImagingDynModel, dyn.GroundStationDynModel):
    # Define some custom properties to be accessed in the state
    @property
    def instrument_pointing_error(self) -> float:
        r_BN_P_unit = self.r_BN_P/np.linalg.norm(self.r_BN_P) 
        c_hat_P = self.satellite.fsw.c_hat_P
        return np.arccos(np.dot(-r_BN_P_unit, c_hat_P))
    
    @property
    def solar_pointing_error(self) -> float:
        a = self.world.gravFactory.spiceObject.planetStateOutMsgs[
            self.world.sun_index
        ].read().PositionVector
        a_hat_N = a / np.linalg.norm(a)
        nHat_B = self.satellite.sat_args["nHat_B"]
        NB = np.transpose(self.BN)
        nHat_N = NB @ nHat_B
        return np.arccos(np.dot(nHat_N, a_hat_N))

class ScanningSatellite(sats.AccessSatellite):
    observation_spec = [
        obs.SatProperties(
            dict(prop="storage_level_fraction"),
            dict(prop="battery_charge_fraction"),
            dict(prop="wheel_speeds_fraction"),
            dict(prop="instrument_pointing_error", norm=np.pi),
            dict(prop="solar_pointing_error", norm=np.pi)
        ),
        obs.OpportunityProperties(
            dict(prop="opportunity_open", norm=5700),
            dict(prop="opportunity_close", norm=5700),
            type="ground_station",
            n_ahead_observe=1,
        ),
        obs.Eclipse(norm=5700),
        obs.Time(),
    ]
    action_spec = [
        act.Scan(duration=180.0),
        act.Charge(duration=120.0),
        act.Downlink(duration=60.0),
        act.Desat(duration=60.0),
    ]
    dyn_type = ScanningDownlinkDynModel
    fsw_type = fsw.ContinuousImagingFSWModel

Next, parameters are set. Since this scenario is focused on maintaining acceptable data
and power levels, these are tuned to create a sufficiently interesting mission.

In [3]:
sat = ScanningSatellite(
    "Scanner-1",
    sat_args=dict(
        # Data
        dataStorageCapacity=5000 * 8e6,  # bits
        storageInit=lambda: np.random.uniform(0.0, 0.8) * 5000 * 8e6,
        instrumentBaudRate=0.5 * 8e6,
        transmitterBaudRate=-50 * 8e6,
        # Power
        batteryStorageCapacity=200 * 3600,  # W*s
        storedCharge_Init=lambda: np.random.uniform(0.3, 1.0) * 200 * 3600,
        basePowerDraw=-10.0,  # W
        instrumentPowerDraw=-30.0,  # W
        transmitterPowerDraw=-25.0,  # W
        thrusterPowerDraw=-80.0,  # W
        panelArea=0.25,
        # Attitude
        imageAttErrorRequirement=0.1,
        imageRateErrorRequirement=0.1,
        disturbance_vector=lambda: np.random.normal(scale=0.0001, size=3),  # N*m
        maxWheelSpeed=6000.0,  # RPM
        wheelSpeeds=lambda: np.random.uniform(-3000, 3000, 3),
        desatAttitude="nadir",
    )
)

Finally, the environment arguments are set. Stepping through this environment is 
demonstrated at the bottom of the page.

In [4]:
duration = 5 * 5700.0  # About 5 orbits
env_args = dict(
    satellite=sat,
    scenario=scene.UniformNadirScanning(value_per_second=1/duration),
    rewarder=data.ScanningTimeReward(),
    time_limit=duration,
    failure_penalty=-1.0,
    terminate_on_time_limit=True,
)

## Set Up Custom Logging

The `bsk_rl` package supplies a utility to make logging information at the end of episodes
easier. This is useful to see how an agent's policy is changing over time, using a
monitoring program such as [TensorBoard](https://www.tensorflow.org/tensorboard). The callback 
is configured by writing a function that takes the environment as an input and returns a
dictionary with values to be logged.

In [5]:
def episode_data_callback(env):
    reward = env.rewarder.cum_reward
    reward = sum(reward.values()) / len(reward)
    orbits = env.simulator.sim_time / (95 * 60)

    data = dict(
        reward=reward,
        # Are satellites dying, and how and when?
        alive=float(env.satellite.is_alive()),
        rw_status_valid=float(env.satellite.dynamics.rw_speeds_valid()),
        battery_status_valid=float(env.satellite.dynamics.battery_valid()),
        orbits_complete=orbits,
    )
    if orbits > 0:
        data["reward_per_orbit"] = reward / orbits
    if not env.satellite.is_alive():
        data["orbits_complete_partial_only"] = orbits
        
    return data

## Configure Ray and PPO

PPO (or some other algorithm) can be configured. Of particular importance
are setting `sample_timeout_s` and `metrics_episode_collection_timeout_s` to appropriately
high values for this environment. The `episode_data_callback` is included in the environment
arguments, and the `WrappedEpisodeDataCallbacks` must be included in training to trigger
logging.

In [6]:
import bsk_rl.utils.rllib  # noqa To access "SatelliteTasking-RLlib"
from ray.rllib.algorithms.ppo import PPOConfig
from bsk_rl.utils.rllib.callbacks import WrappedEpisodeDataCallbacks

N_CPUS = 3

training_args = dict(
    lr=0.00003,
    gamma=0.999,
    train_batch_size=250,  # usually a larger number, like 2500
    num_sgd_iter=10,
    model=dict(fcnet_hiddens=[512, 512], vf_share_layers=False),
    lambda_=0.95,
    use_kl_loss=False,
    clip_param=0.1,
    grad_clip=0.5,
)

config = (
    PPOConfig()
    .training(**training_args)
    .env_runners(num_env_runners=N_CPUS-1, sample_timeout_s=1000.0)
    .environment(
        env="SatelliteTasking-RLlib",
        env_config=dict(**env_args, episode_data_callback=episode_data_callback),
    )
    .reporting(
        metrics_num_episodes_for_smoothing=1,
        metrics_episode_collection_timeout_s=180,
    )
    .checkpointing(export_native_model_files=True)
    .framework(framework="torch")
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .callbacks(WrappedEpisodeDataCallbacks)
)

Once the PPO configuration has been set, `ray` can be started and the agent can be
trained.

Training on a reasonably modern machine, we can achieve 5M steps over 32 processors in 6
to 18 hours, depending on specific environment configurations.

Note that the custom logging metrics are reported under env_runners.

In [7]:
import ray
from ray import tune

ray.init(
    ignore_reinit_error=True,
    num_cpus=N_CPUS,
    object_store_memory=2_000_000_000,  # 2 GB
)

# Run the training
tune.run(
    "PPO",
    config=config.to_dict(),
    stop={"training_iteration": 10},  # Adjust the number of iterations as needed
    checkpoint_freq=10,
    checkpoint_at_end=True
)

# Shutdown Ray
ray.shutdown()

2024-09-12 15:06:17,569	INFO worker.py:1783 -- Started a local Ray instance.


2024-09-12 15:06:17,819	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


(PPO pid=96080) Install gputil for GPU system monitoring.


Trial name,env_runners,fault_tolerance,learners,num_agent_steps_sampled_lifetime,num_env_steps_sampled_lifetime,num_env_steps_trained_lifetime,num_episodes_lifetime,perf,timers
PPO_SatelliteTasking-RLlib_da747_00000,"{'num_episodes': 2, 'num_module_steps_sampled': {'default_policy': 250}, 'reward': 0.40487192982456155, 'sample': 1.2900817312067883, 'num_module_steps_sampled_lifetime': {'default_policy': 13750}, 'orbits_complete': 4.957157894736842, 'num_env_steps_sampled': 250, 'num_agent_steps_sampled': {'default_agent': 250}, 'rw_status_valid': 1.0, 'battery_status_valid': 0.99, 'num_env_steps_sampled_lifetime': 25000, 'num_agent_steps_sampled_lifetime': {'default_agent': 13750}, 'alive': 0.99, 'module_episode_returns_mean': {'default_policy': -0.7653859649122806}, 'episode_len_mean': 135.0, 'reward_per_orbit': 0.08157333539731684, 'episode_len_min': 36, 'episode_return_max': -0.5808070175438594, 'agent_episode_returns_mean': {'default_agent': -0.7653859649122806}, 'episode_len_max': 234, 'orbits_complete_partial_only': 0.7157894736842106, 'time_between_sampling': 0.11275066755158615, 'episode_return_mean': -0.7653859649122806, 'episode_duration_sec_mean': 1.4712897290301044, 'episode_return_min': -0.9499649122807018}","{'num_healthy_workers': 2, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}","{'default_policy': {'curr_entropy_coeff': 0.0, 'vf_explained_var': 0.19891756772994995, 'num_module_steps_trained': 250, 'vf_loss_unclipped': 0.04255196452140808, 'policy_loss': 0.26024314761161804, 'num_non_trainable_parameters': 0.0, 'gradients_default_optimizer_global_norm': 0.5620529651641846, 'default_optimizer_learning_rate': 3e-05, 'num_trainable_parameters': 139525.0, 'mean_kl_loss': 0.0, 'total_loss': 0.3027951121330261, 'vf_loss': 0.04255196452140808, 'entropy': 1.353834629058838}, '__all_modules__': {'num_module_steps_trained': 250, 'num_non_trainable_parameters': 0.0, 'total_loss': 0.3027951121330261, 'num_trainable_parameters': 139525.0, 'num_env_steps_trained': 250}}",{'default_agent': 2500},2500,2500,12,"{'cpu_util_percent': 23.9, 'ram_util_percent': 84.4}","{'env_runner_sampling_timer': 1.3166690483695032, 'learner_update_timer': 0.06828840010371885, 'synch_weights': 0.0038879869585440557, 'synch_env_connectors': 0.005117139493842671}"


...

(SingleAgentEnvRunner pid=96083) 2024-09-12 15:06:35,744 sats.satellite.Scanner-1       WARNING    <27840.00> Scanner-1: failed battery_valid check [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(SingleAgentEnvRunner pid=96083) 2024-09-12 15:06:41,033 sats.satellite.Scanner-1       WARNING    <26460.00> Scanner-1: failed battery_valid check [repeated 3x across cluster]


(PPO pid=96080) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/markstephenson/ray_results/PPO_2024-09-12_15-06-17/PPO_SatelliteTasking-RLlib_da747_00000_0_2024-09-12_15-06-17/checkpoint_000000)
2024-09-12 15:06:43,328	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/markstephenson/ray_results/PPO_2024-09-12_15-06-17' in 0.0151s.


2024-09-12 15:06:43,587	INFO tune.py:1041 -- Total run time: 25.77 seconds (25.47 seconds for the tuning loop).


(SingleAgentEnvRunner pid=96082) 2024-09-12 15:06:42,194 sats.satellite.Scanner-1       WARNING    <4080.00> Scanner-1: failed battery_valid check


## Loading the Policy Network

The policy network can be found in the `p0` subdirectory of the checkpoint output, if 
using the torch backend, and the `model` subdirectory of the checkpoint output. Use
`bsk_rl.utils.rllib.load_torch_mlp_policy` to load torch policies.

## Stepping Through the Environment

The environment is stepped through with random actions to give a sense of how it acts.

In [8]:
from bsk_rl import SatelliteTasking

env = SatelliteTasking(**env_args, log_level="INFO")
env.reset()
terminated = False
while not terminated:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

2024-09-12 15:06:45,134 gym                            INFO       Resetting environment with seed=304261736


2024-09-12 15:06:45,217 sats.satellite.Scanner-1       INFO       <0.00> Scanner-1: Finding opportunity windows from 0.00 to 28500.00 seconds


2024-09-12 15:06:45,283 gym                            INFO       <0.00> Environment reset


2024-09-12 15:06:45,283 gym                            INFO       <0.00> === STARTING STEP ===


2024-09-12 15:06:45,283 sats.satellite.Scanner-1       INFO       <0.00> Scanner-1: action_nadir_scan tasked for 180.0 seconds


...

2024-09-12 15:06:47,579 sats.satellite.Scanner-1       INFO       <22800.00> Scanner-1: Satellite Scanner-1 requires retasking


2024-09-12 15:06:47,579 sats.satellite.Scanner-1       WARNING    <22800.00> Scanner-1: failed battery_valid check


2024-09-12 15:06:47,580 gym                            INFO       <22800.00> Step reward: -0.9950877192982456


2024-09-12 15:06:47,580 gym                            INFO       <22800.00> Episode terminated: True


2024-09-12 15:06:47,580 gym                            INFO       <22800.00> Episode truncated: False
